In [ ]:
import pandas as pd


In [3]:
import openpyxl
import pandas as pd
from datetime import datetime, date

# Define the file path to your Excel file
file_path = "/content/2024 E-Newsletters.xlsx"

# Load the workbook using openpyxl
wb = openpyxl.load_workbook(file_path, data_only=True)  # Use data_only=True to get cell values instead of formulas

# Get all sheet names
sheet_names = wb.sheetnames

# Sheets to remove
sheets_to_remove = ['Tomorrows Outlook', 'Remaining Full Year Openings', 'Future Outlook', 'Todays Overview']

# Initialize a list to store all data
all_data = []

# Function to check if a cell has a red fill color in the first column
def is_red_fill(cell):
    if cell.column == 1 and cell.fill.fill_type == "solid" and cell.fill.start_color.rgb == "FFFF0000":
        return True
    return False

# Function to check if a cell has red or black font color
def is_coloured_cell(cell):
    if cell.value is not None and cell.font.color and cell.font.color.rgb in ["FFFF0000", "FF000000"]:
        return True
    return False

# Iterate through each sheet
for sheet_name in sheet_names:
    if sheet_name not in sheets_to_remove:
        # Access the sheet
        sheet = wb[sheet_name]

        # Initialize variables for headers and data
        headers = None

        # Iterate through rows
        for i, row in enumerate(sheet.iter_rows()):
            if i == 0:
                # First row as headers
                headers = [cell.value for cell in row[:9]]  # Assuming you want the first 9 columns as headers
            else:
                # Check if the first column contains a valid date
                first_cell = row[0]
                if isinstance(first_cell.value, (datetime, date)):
                    skip_row = False
                    # Check if any cell in the row is red-filled in the first column
                    for cell in row[:1]:  # Checking only the first column
                        if is_red_fill(cell):
                            skip_row = True
                            break

                    if not skip_row:
                        # Iterate through each cell in the row (excluding the first column)
                        for j, cell in enumerate(row[1:9], start=1):
                            if is_coloured_cell(cell):
                                all_data.append({
                                    'Coordinates': cell.coordinate,
                                    'Date': first_cell.value,
                                    'Data': cell.value,
                                    'Column Name': headers[j],
                                    'Sheet Name': sheet_name
                                })

# Create a Pandas DataFrame from all collected data
df = pd.DataFrame(all_data)

# Close the workbook
wb.close()

# Optionally, you can save the DataFrame to a CSV file or further process it as needed
# df.to_csv('output.csv', index=False)


In [4]:
df

,Coordinates,Date,Data,Column Name,Sheet Name
0,C4,2024-01-02,American College of Cardiology / ACC (MB) 12.2...,Ad Position 1,Hospital
1,C5,2024-01-03,AstraZeneca (Andexxa) (MT/SS)\n12.20.2023\n\nh...,Ad Position 1,Hospital
2,C6,2024-01-04,J.D. Power (MB)\n11.30.2023\n\nhttps://cdn.ome...,Ad Position 1,Hospital
3,D6,2024-01-04,HWL (MP/CX)(1.3.2023)\n\nhttps://cdn.omeda.com...,Ad Position 2,Hospital
4,E6,2024-01-04,BPD (Somer) 12.21.2023\n\nhttps://cdn.omeda.co...,Ad Position 3,Hospital
...,...,...,...,...,...
2648,C95,2024-11-20,Merck V116 (AE/SS),Ad Position 1,Physician Leadership
2649,C97,2024-11-27,Merck V116 (AE/SS),Ad Position 1,Physician Leadership
2650,C99,2024-12-04,Merck V116 (AE/SS),Ad Position 1,Physician Leadership
2651,C100,2024-12-06,Merck VXN (AE/SS)_02.22.2024\n\nhttps://cdn.om...,Ad Position 1,Physician Leadership


In [73]:
import re

def extract_second_url(text):
    urls = re.findall(r'(https?://\S+)', text)
    if len(urls) >= 2:
        return urls[1]  # Select the second URL if available
    elif len(urls) == 1:
        return urls[0]  # Select the first (and only) URL if there's only one
    else:
        return None  # Return None if no URLs are found

In [81]:
# Extracting required information
df['client manager'] = df['Data'].str.extract(r'\(([^0-9]+)\)')
df['client'] = df['Data'].str.extract(r'^(.*?)\s*\(')
df['date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.strftime('%m.%d.%Y')
df['sheet_to_a'] = df['Sheet Name'] + ' e-news letter ' + df['Column Name']
df['platform'] = 'Omeda'
df['notes'] = 'standard-date, impressions, clicks, CTR - '+ df['Data'].apply(extract_second_url)

# Selecting required columns
selected_columns = ['Coordinates', 'client manager', 'client', 'date', 'sheet_to_a', 'platform', 'notes']
selected_df = df[selected_columns]


In [82]:
selected_df

,Coordinates,client manager,client,date,sheet_to_a,platform,notes
0,C4,MB,American College of Cardiology / ACC,01.02.2024,Hospital e-news letter Ad Position 1,Omeda,"standard-date, impressions, clicks, CTR - http..."
1,C5,Andexxa) (MT/SS,AstraZeneca,01.03.2024,Hospital e-news letter Ad Position 1,Omeda,"standard-date, impressions, clicks, CTR - http..."
2,C6,MB,J.D. Power,01.04.2024,Hospital e-news letter Ad Position 1,Omeda,"standard-date, impressions, clicks, CTR - http..."
3,D6,MP/CX,HWL,01.04.2024,Hospital e-news letter Ad Position 2,Omeda,"standard-date, impressions, clicks, CTR - http..."
4,E6,Somer,BPD,01.04.2024,Hospital e-news letter Ad Position 3,Omeda,"standard-date, impressions, clicks, CTR - http..."
...,...,...,...,...,...,...,...
2648,C95,AE/SS,Merck V116,11.20.2024,Physician Leadership e-news letter Ad Position 1,Omeda,NaN
2649,C97,AE/SS,Merck V116,11.27.2024,Physician Leadership e-news letter Ad Position 1,Omeda,NaN
2650,C99,AE/SS,Merck V116,12.04.2024,Physician Leadership e-news letter Ad Position 1,Omeda,NaN
2651,C100,AE/SS,Merck VXN,12.06.2024,Physician Leadership e-news letter Ad Position 1,Omeda,"standard-date, impressions, clicks, CTR - http..."


In [83]:

selected_df['date'] = pd.to_datetime(selected_df['date'])  # Convert 'Date' column to datetime format if needed

# Get current month and year
current_month = datetime.now().month
current_year = datetime.now().year

# Filter rows for current month
current_month_data = selected_df[(selected_df['date'].dt.month == current_month) & (selected_df['date'].dt.year == current_year)]

current_month_data

<ipython-input-83-aca8ae9be69d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['date'] = pd.to_datetime(selected_df['date'])  # Convert 'Date' column to datetime format if needed


,Coordinates,client manager,client,date,sheet_to_a,platform,notes
284,C157,CX/LE/MP,T-Mobile,2024-06-01,Hospital e-news letter Ad Position 1,Omeda,"standard-date, impressions, clicks, CTR - http..."
285,D157,CX/LE/MP,T-Mobile,2024-06-01,Hospital e-news letter Ad Position 2,Omeda,"standard-date, impressions, clicks, CTR - http..."
286,E157,CX/LE/MP,T-Mobile,2024-06-01,Hospital e-news letter Ad Position 3,Omeda,"standard-date, impressions, clicks, CTR - http..."
287,F157,CX/LE/MP,T-Mobile,2024-06-01,Hospital e-news letter Text Ad,Omeda,"standard-date, impressions, clicks, CTR - http..."
288,C158,Somer,Staples,2024-06-02,Hospital e-news letter Ad Position 1,Omeda,"standard-date, impressions, clicks, CTR - http..."
...,...,...,...,...,...,...,...
2606,D51,MT/Jrod,Mayo Clinic Laboratories,2024-06-19,Physician Leadership e-news letter Ad Position 2,Omeda,"standard-date, impressions, clicks, CTR - http..."
2607,E51,MT/Jrod,Mayo Clinic Laboratories,2024-06-19,Physician Leadership e-news letter Ad Position 3,Omeda,"standard-date, impressions, clicks, CTR - http..."
2608,F51,MT/Jrod,Mayo Clinic Laboratories,2024-06-19,Physician Leadership e-news letter Text Ad,Omeda,"standard-date, impressions, clicks, CTR - http..."
2609,C53,AE/SS,Merck VXN,2024-06-26,Physician Leadership e-news letter Ad Position 1,Omeda,"standard-date, impressions, clicks, CTR - http..."


In [77]:
# Save filtered data to Excel
file_path = f"current_month_data_{current_month}_{current_year}.xlsx"
current_month_data.to_excel(file_path, index=False)